# spark初始化

In [61]:
%%spark.set_config

spark.executor.instances=1
spark.executor.memory=512m
spark.executor.cores=1

In [62]:
!echo $SPARK_HOME

/opt/apps/SPARK3/spark3-current


In [63]:
spark, sc = %spark.get_session PeiSiyu

24/05/25 15:35:05 WARN [Thread-351] Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


# 加载数据

In [64]:
movies = spark.read.options(header='true', inferSchema='true').csv("/user/peisiyu/movies.csv") 
movies.show()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

In [ ]:
tags = spark.read.options(header='true', inferSchema='true').csv("/user/peisiyu/tags.csv") 
tags.show()

In [ ]:
links = spark.read.options(header='true', inferSchema='true').csv("/user/peisiyu/links.csv") 
links.show()

In [65]:
ratings = spark.read.options(header='true', inferSchema='true').csv("/user/peisiyu/ratings.csv") 
ratings.show()

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
|     1|     70|   3.0|964982400|
|     1|    101|   5.0|964980868|
|     1|    110|   4.0|964982176|
|     1|    151|   5.0|964984041|
|     1|    157|   5.0|964984100|
|     1|    163|   5.0|964983650|
|     1|    216|   5.0|964981208|
|     1|    223|   3.0|964980985|
|     1|    231|   5.0|964981179|
|     1|    235|   4.0|964980908|
|     1|    260|   5.0|964981680|
|     1|    296|   3.0|964982967|
|     1|    316|   3.0|964982310|
|     1|    333|   5.0|964981179|
|     1|    349|   4.0|964982563|
+------+-------+------+---------+
only showing top 20 rows



In [ ]:
tags = spark.read.options(header='true', inferSchema='true').csv("/user/peisiyu/tags.csv") 
tags.show()

# 正式开始大作业的内容

# LFM矩阵分解算法

## 安装需要的包

In [28]:
%%spark.make_env
random
#好像没用上

Installing requirements...
Looking in indexes: http://mirrors.cloud.aliyuncs.com/pypi/simple
ERROR: Could not find a version that satisfies the requirement random (from versions: none)
ERROR: No matching distribution found for random
Looking in indexes: http://mirrors.cloud.aliyuncs.com/pypi/simple
ERROR: Could not find a version that satisfies the requirement random (from versions: none)
ERROR: No matching distribution found for random
Install requirements finished
Put env to hdfs...
Put env finished


In [29]:
import numpy as np
import pandas as pd
import random

## 提取评分数据ratings的前三列（即除了时间戳以外的三列）

In [66]:
new_ratings = ratings[['userId', 'movieId', 'rating']]

In [67]:
new_ratings.show()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|      1|   4.0|
|     1|      3|   4.0|
|     1|      6|   4.0|
|     1|     47|   5.0|
|     1|     50|   5.0|
|     1|     70|   3.0|
|     1|    101|   5.0|
|     1|    110|   4.0|
|     1|    151|   5.0|
|     1|    157|   5.0|
|     1|    163|   5.0|
|     1|    216|   5.0|
|     1|    223|   3.0|
|     1|    231|   5.0|
|     1|    235|   4.0|
|     1|    260|   5.0|
|     1|    296|   3.0|
|     1|    316|   3.0|
|     1|    333|   5.0|
|     1|    349|   4.0|
+------+-------+------+
only showing top 20 rows



In [68]:
#划分训练集和测试集

# 指定训练集和测试集的权重
weights = [0.8, 0.2]  # 训练集权重为0.8，测试集权重为0.2

# 使用randomSplit方法拆分DataFrame
split_data = new_ratings.randomSplit(weights, seed=42)

# 获取拆分后的训练集和测试集
training = split_data[0]
test = split_data[1]

In [70]:
training.show()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|      1|   4.0|
|     1|      3|   4.0|
|     1|     47|   5.0|
|     1|     50|   5.0|
|     1|     70|   3.0|
|     1|    110|   4.0|
|     1|    157|   5.0|
|     1|    163|   5.0|
|     1|    216|   5.0|
|     1|    223|   3.0|
|     1|    235|   4.0|
|     1|    260|   5.0|
|     1|    296|   3.0|
|     1|    316|   3.0|
|     1|    333|   5.0|
|     1|    356|   4.0|
|     1|    362|   5.0|
|     1|    367|   4.0|
|     1|    441|   4.0|
|     1|    457|   5.0|
+------+-------+------+
only showing top 20 rows



In [71]:
test.show()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|      6|   4.0|
|     1|    101|   5.0|
|     1|    151|   5.0|
|     1|    231|   5.0|
|     1|    349|   4.0|
|     1|    423|   3.0|
|     1|    543|   4.0|
|     1|    596|   5.0|
|     1|    923|   5.0|
|     1|    940|   5.0|
|     1|    943|   4.0|
|     1|   1009|   3.0|
|     1|   1024|   5.0|
|     1|   1031|   5.0|
|     1|   1089|   5.0|
|     1|   1197|   5.0|
|     1|   1220|   5.0|
|     1|   1573|   5.0|
|     1|   1805|   4.0|
|     1|   1967|   4.0|
+------+-------+------+
only showing top 20 rows



In [79]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [95]:
# 创建 ALS 模型对象并设置超参数
als = ALS(
    userCol='userId',
    itemCol='movieId',
    ratingCol='rating',
    coldStartStrategy='drop',  # 处理冷启动问题，直接将缺失值的数据项丢弃
    rank=50,  # 设置秩（rank）超参数为10
    maxIter=10,  # 设置迭代次数（maxIter）超参数为10
    regParam=0.01  # 设置正则化参数（regParam）超参数为0.1
)

In [96]:
# 使用训练集进行模型训练
model = als.fit(training)

In [97]:
# 使用测试集进行模型评估
predictions = model.transform(test)

In [98]:
# 评估指标选择 RMSE（均方根误差）
evaluator = RegressionEvaluator(
    metricName='rmse',
    labelCol='rating',
    predictionCol='prediction'
)


In [99]:
# 计算 RMSE
rmse = evaluator.evaluate(predictions)
print('Root Mean Squared Error (RMSE) =', rmse)


Root Mean Squared Error (RMSE) = 1.3322243297855718


In [101]:
# 为用户生成 Top-N 推荐结果
userRecs = model.recommendForAllUsers(10)  # 生成每个用户的 Top-10 推荐结果

In [102]:
userRecs.show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|     1|[{924, 5.775116},...|
|     3|[{6835, 5.0248733...|
|     5|[{4993, 6.37119},...|
|     6|[{1287, 5.1129436...|
|     9|[{48780, 5.580054...|
|    12|[{63082, 5.891697...|
|    13|[{1240, 5.38916},...|
|    15|[{924, 6.4105315}...|
|    16|[{16, 5.5438595},...|
|    17|[{4002, 5.361268}...|
|    19|[{3477, 5.037508}...|
|    20|[{1641, 5.7160797...|
|    22|[{1097, 7.9977565...|
|    26|[{1527, 4.3192067...|
|    27|[{915, 5.4210052}...|
|    28|[{53123, 5.047100...|
|    31|[{61240, 5.495505...|
|    34|[{58998, 5.411299...|
|    35|[{924, 6.1741886}...|
|    37|[{1407, 5.9614167...|
+------+--------------------+
only showing top 20 rows



# 这一部分不要了

In [74]:
#透视数据
data = training.groupBy("userId").pivot("movieId").agg({"rating": "first"})
data = data.na.fill(0)

In [75]:
type(data)

pyspark.sql.dataframe.DataFrame

In [76]:
#提取行索引和列索引
data_index = data.select("userId").rdd.flatMap(lambda x: x).collect()
data_columns = data.columns[1:]  #排除userid列

# 关闭开启的spark任务

In [60]:
%spark.stop_session

'success stop spark application application_1709108427864_1855'